# Ryan messing around with lakeFS
Hoo boy I spent hours messing with this on SageMaker. Got nowhere trying to even query my repo. This appears to go smoother locally. I'll just chalk it up to my own inexperience on AWS. Something tells me that AWS will have their own data versioning service in the future...

LakeFS documentation here: https://docs.lakefs.io/v0.86/integrations/python.html

Anyway let's get the client instatiated:

In [2]:
import lakefs_client
from lakefs_client import models
from lakefs_client.client import LakeFSClient

# lakeFS credentials and endpoint
configuration = lakefs_client.Configuration()
configuration.username = ''
configuration.password = ''
configuration.host = ''

client = LakeFSClient(configuration)

Kay cool. Let's query the countries repo, which I had previously created and committed to. 

In [3]:
client.branches.list_branches('countries')

{'pagination': {'has_more': False,
                'max_per_page': 1000,
                'next_offset': '',
                'results': 1},
 'results': [{'commit_id': 'cb3faebd8422a233e7d39739876167ad0a21e919c5cf7bff73d46d1d26b9089c',
              'id': 'main'}]}

That looks right, since I only have the main branch in there. Let's fork that one and work inside of it.

In [4]:
client.branches.create_branch(repository='countries', branch_creation=models.BranchCreation(name='main-fork', source='main'))

'cb3faebd8422a233e7d39739876167ad0a21e919c5cf7bff73d46d1d26b9089c'

In [5]:
client.branches.list_branches('countries')

{'pagination': {'has_more': False,
                'max_per_page': 1000,
                'next_offset': '',
                'results': 2},
 'results': [{'commit_id': 'cb3faebd8422a233e7d39739876167ad0a21e919c5cf7bff73d46d1d26b9089c',
              'id': 'main'},
             {'commit_id': 'cb3faebd8422a233e7d39739876167ad0a21e919c5cf7bff73d46d1d26b9089c',
              'id': 'main-fork'}]}

Well that looks right. Let's see if we can grab the data out of the newly-forked branch.

In [7]:
client.objects.list_objects('countries', 'main-fork')

{'pagination': {'has_more': False,
                'max_per_page': 1000,
                'next_offset': '',
                'results': 0},
 'results': []}

Ah shit. I did not fork it and merely created an empty branch. Let's try something different.

In [8]:
client.branches.delete_branch('countries', 'main-fork')

In [9]:
client.branches.list_branches('countries')

{'pagination': {'has_more': False,
                'max_per_page': 1000,
                'next_offset': '',
                'results': 1},
 'results': [{'commit_id': 'cb3faebd8422a233e7d39739876167ad0a21e919c5cf7bff73d46d1d26b9089c',
              'id': 'main'}]}

Deleted the empty branch... what to do now... Documentation could be better for us noobs, but I digress.

Well shit. Looks like I'll manually have to pull the data from the main branch over to the new branch after all...

In [10]:
client.branches.create_branch(repository='countries', branch_creation=models.BranchCreation(name='working', source='main'))

'cb3faebd8422a233e7d39739876167ad0a21e919c5cf7bff73d46d1d26b9089c'

In [14]:
client.branches.list_branches('countries')

{'pagination': {'has_more': False,
                'max_per_page': 1000,
                'next_offset': '',
                'results': 2},
 'results': [{'commit_id': 'cb3faebd8422a233e7d39739876167ad0a21e919c5cf7bff73d46d1d26b9089c',
              'id': 'main'},
             {'commit_id': 'cb3faebd8422a233e7d39739876167ad0a21e919c5cf7bff73d46d1d26b9089c',
              'id': 'working'}]}

*** DELETED WHINING ***

In [17]:
client.objects.list_objects('countries','main')

{'pagination': {'has_more': False,
                'max_per_page': 1000,
                'next_offset': '',
                'results': 1},
 'results': [{'checksum': '13b1d123f0cd1716b1245eaacd4e770a',
              'content_type': 'text/csv',
              'mtime': 1701367563,
              'path': 'data.csv',
              'path_type': 'object',
              'physical_address': 's3://ryans-lakefs-test/countries/data/gjj380if4ngu7qoas2bg/clkctvif4ngu7qoas2c0',
              'size_bytes': 16664919}]}

In [39]:
client.objects.copy_object('countries','working','data.csv',
                           ObjectCopyCreation(
                            src_path='data.csv',
                            src_ref='main')
                          )

{'checksum': '13b1d123f0cd1716b1245eaacd4e770a',
 'content_type': 'text/csv',
 'mtime': 1701384696,
 'path': 'data.csv',
 'path_type': 'object',
 'physical_address': 's3://ryans-lakefs-test/countries/data/gjiv2o2f4ngu7qoas2hg/clkh3u2f4ngu7qoas2ig',
 'size_bytes': 16664919}

Mother fuck. Not noob friendly. At least someone gets the example notebook... that I needed. Now that I have the data copied over from the main branch into the working branch, let's see if we can get this csv into pandas.

In [50]:
file = client.objects.get_object('countries','working','data.csv')

In [51]:
file

<_io.BufferedReader name='/tmp/tmp1xb08017'>

The hell am I supposed to do with this?!

In [ ]:
# file.read()
# outputs the entire csv here as a raw string

Hmmm... how do I get that into pandas...

In [52]:
import pandas as pd
df = pd.read_csv(file)

In [53]:
df

,CID,FID_countries_points,FID_L0World_Countries_Generalized,COUNTRY,ISO,COUNTRYAFF,AFF_ISO,Longitude,Latitude
0,500,NaN,NaN,Ocean,NaN,NaN,NaN,-30.044039,24.216976
1,500,NaN,NaN,Ocean,NaN,NaN,NaN,150.388214,-21.728706
2,500,NaN,NaN,Ocean,NaN,NaN,NaN,168.733444,56.352604
3,500,NaN,NaN,Ocean,NaN,NaN,NaN,-34.896107,4.820187
4,500,NaN,NaN,Ocean,NaN,NaN,NaN,3.984333,-26.328730
...,...,...,...,...,...,...,...,...,...
260994,91,90847.0,91.0,Greenland,GL,Denmark,DK,-27.684013,83.460632
260995,91,90419.0,91.0,Greenland,GL,Denmark,DK,-29.777115,83.461273
260996,91,90501.0,91.0,Greenland,GL,Denmark,DK,-33.087837,83.464043
260997,91,90074.0,91.0,Greenland,GL,Denmark,DK,-33.384373,83.475655


Hell yeah! It only took an entire day... and I was ready to call it...

In [57]:
cleaned_df = df.drop(['FID_countries_points',
         'FID_L0World_Countries_Generalized',
         'COUNTRY',
         'COUNTRYAFF',
         'AFF_ISO',
         'ISO'
        ], axis=1)

cleaned_df

,CID,Longitude,Latitude
0,500,-30.044039,24.216976
1,500,150.388214,-21.728706
2,500,168.733444,56.352604
3,500,-34.896107,4.820187
4,500,3.984333,-26.328730
...,...,...,...
260994,91,-27.684013,83.460632
260995,91,-29.777115,83.461273
260996,91,-33.087837,83.464043
260997,91,-33.384373,83.475655


So scikit-learn should like that. There is other stuff I could do, but let's keep it simple since we are not really worried about model accuracy. Let's make another dataframe with the CID and the country, which we will need to interpret the model prediction.

In [64]:
country_keys = df.drop([
        'FID_countries_points',
        'COUNTRYAFF',
        'AFF_ISO',
        'ISO',
        'Longitude',
        'Latitude',
        'FID_L0World_Countries_Generalized'
        ], axis=1).drop_duplicates().sort_values('CID')

In [65]:
country_keys

,CID,COUNTRY
177625,1,Afghanistan
202167,2,Albania
154943,3,Algeria
44461,4,American Samoa
212777,5,Andorra
...,...,...
44806,248,Wallis and Futuna
114193,249,Yemen
37425,250,Zambia
27788,251,Zimbabwe


Looks like the day might not be a waste after all... let's get these bad boys back into the working branch of the lakeFS repo.

In [69]:
# https://stackoverflow.com/questions/52089872/convert-pandas-dataframe-to-bytes-like-object
# credit to Iain Hunter
from io import BytesIO  

client.objects.upload_object('countries','working','data.csv', 
                             content=BytesIO(cleaned_df.to_csv(index=False).encode('utf-8')))

{'checksum': '3ea72619187403ade62f4eb694f90b74',
 'content_type': 'application/octet-stream',
 'mtime': 1701388581,
 'path': 'data.csv',
 'path_type': 'object',
 'physical_address': 's3://ryans-lakefs-test/countries/data/gjiu3o2f4ngu7qoas2j0/clki282f4ngu7qoas2jg',
 'size_bytes': 6754629}

That part was relatively painless... just checked in the GUI and the csv has been updated. Let's do ther other one now as well.

In [70]:
client.objects.upload_object('countries','working','countries.csv', 
                             content=BytesIO(country_keys.to_csv(index=False).encode('utf-8')))

{'checksum': '9d8ac4c86f669711a1fd35633d5c6f97',
 'content_type': 'application/octet-stream',
 'mtime': 1701388924,
 'path': 'countries.csv',
 'path_type': 'object',
 'physical_address': 's3://ryans-lakefs-test/countries/data/gjiu3o2f4ngu7qoas2j0/clki4uqf4ngu7qoas2k0',
 'size_bytes': 3609}

In [71]:
client.objects.list_objects('countries','working')

{'pagination': {'has_more': False,
                'max_per_page': 1000,
                'next_offset': '',
                'results': 2},
 'results': [{'checksum': '9d8ac4c86f669711a1fd35633d5c6f97',
              'content_type': 'application/octet-stream',
              'mtime': 1701388924,
              'path': 'countries.csv',
              'path_type': 'object',
              'physical_address': 's3://ryans-lakefs-test/countries/data/gjiu3o2f4ngu7qoas2j0/clki4uqf4ngu7qoas2k0',
              'size_bytes': 3609},
             {'checksum': '3ea72619187403ade62f4eb694f90b74',
              'content_type': 'application/octet-stream',
              'mtime': 1701388581,
              'path': 'data.csv',
              'path_type': 'object',
              'physical_address': 's3://ryans-lakefs-test/countries/data/gjiu3o2f4ngu7qoas2j0/clki282f4ngu7qoas2jg',
              'size_bytes': 6754629}]}

Yup yup let's commit this then merge it back into the main branch.

In [73]:
client.branches_api.diff_branch(repository='countries', branch='working')

{'pagination': {'has_more': False,
                'max_per_page': 1000,
                'next_offset': '',
                'results': 2},
 'results': [{'path': 'countries.csv',
              'path_type': 'object',
              'size_bytes': 3609,
              'type': 'added'},
             {'path': 'data.csv',
              'path_type': 'object',
              'size_bytes': 6754629,
              'type': 'added'}]}

In [75]:
from lakefs_client.model.commit_creation import CommitCreation

client.commits_api.commit('countries', 'working', 
                         CommitCreation(message='Updated data.csv, added countries.csv')
                         )

{'committer': 'test-user',
 'creation_date': 1701389508,
 'id': '200a6679e879047bdd2225ac91a951ab39bd74356dc8471233f5d4ab99573a46',
 'message': 'Updated data.csv, added countries.csv',
 'meta_range_id': '',
 'metadata': {},
 'parents': ['cb3faebd8422a233e7d39739876167ad0a21e919c5cf7bff73d46d1d26b9089c']}

In [76]:
client.branches_api.diff_branch(repository='countries', branch='working')

{'pagination': {'has_more': False,
                'max_per_page': 1000,
                'next_offset': '',
                'results': 0},
 'results': []}

Huzzah! Now let's merge that back into the main branch.

In [77]:
client.refs_api.diff_refs(repository='countries', left_ref='main', right_ref='working')

{'pagination': {'has_more': False,
                'max_per_page': 1000,
                'next_offset': '',
                'results': 2},
 'results': [{'path': 'countries.csv',
              'path_type': 'object',
              'size_bytes': 3609,
              'type': 'added'},
             {'path': 'data.csv',
              'path_type': 'object',
              'size_bytes': 6754629,
              'type': 'added'}]}

In [79]:
client.refs_api.merge_into_branch(repository='countries', source_ref='working', destination_branch='main')

{'reference': '2975cc6d3d94c256ee529d0d7048bee22874c7d70f8a0cea011df3a8d70c7ee8'}

In [80]:
client.refs_api.diff_refs(repository='countries', left_ref='main', right_ref='working')

{'pagination': {'has_more': False,
                'max_per_page': 1000,
                'next_offset': '',
                'results': 0},
 'results': []}

In [81]:
client.objects.list_objects('countries','main')

{'pagination': {'has_more': False,
                'max_per_page': 1000,
                'next_offset': '',
                'results': 2},
 'results': [{'checksum': '9d8ac4c86f669711a1fd35633d5c6f97',
              'content_type': 'application/octet-stream',
              'mtime': 1701388924,
              'path': 'countries.csv',
              'path_type': 'object',
              'physical_address': 's3://ryans-lakefs-test/countries/data/gjiu3o2f4ngu7qoas2j0/clki4uqf4ngu7qoas2k0',
              'size_bytes': 3609},
             {'checksum': '3ea72619187403ade62f4eb694f90b74',
              'content_type': 'application/octet-stream',
              'mtime': 1701388581,
              'path': 'data.csv',
              'path_type': 'object',
              'physical_address': 's3://ryans-lakefs-test/countries/data/gjiu3o2f4ngu7qoas2j0/clki282f4ngu7qoas2jg',
              'size_bytes': 6754629}]}

# Conclusion

And there you have it! Through struggle, we created a new branch, copied over the contents from the main branch, did work on it locally, uploaded and committed the changes to the new branch, then merged the new branch and the main branch. 

Since we are using version control on the data, potential conflicts with others working on the data could be avoided. Similarly, reverting data should anything go wrong is totally possible. This is some sort of MLOps best practice implemented in a sandbox of sorts. A real production implementation would look far different, I am sure.